<h>Spark ML <h>

In [1]:
! pip install pyspark

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer # Perform encoding
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier





In [7]:
#Create a Spark session
spark = SparkSession.builder \
    .appName("NeuralNetworkExample") \
    .getOrCreate()




RuntimeError: Java gateway process exited before sending its port number

In [ ]:
spark

In [ ]:
# Read the dataset into a DataFrame
titanic_df = spark.read.parquet("train.parquet")

# Load your dataset into a Spark DataFrame
data = spark.read.csv("your_data.csv", header=True, inferSchema=True)

# Prepare features vector
feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)


train_data, test_data = data.randomSplit([0.8, 0.2], seed=123)


# Specify the layers of the neural network
layers = [len(feature_columns), 10, 5, 2]  # Example architecture: input layer, two hidden layers, output layer

# Define the neural network classifier
nn_classifier = MultilayerPerceptronClassifier(layers=layers, seed=123)

# Train the model
model = nn_classifier.fit(train_data)


# Make predictions on test data
predictions = model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy:", accuracy)



root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Family_Size: integer (nullable = true)
 |-- Alone: integer (nullable = true)

+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Family_Size|Alone|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|          1|    0|
|       1|     1|female|38.0|    1|    0|71.2833|       C|          1|    0|
|       1|     3|female|26.0|    0|    0|  7.925|       S|          0|    1|
|       1|     1|female|35.0|    1|    0|   53.1|       S|          1|    0|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|          0| 

In [ ]:

#Выведите значения датафрейма с помощью метода show()
titanic_df.show()

# Мы будем классифицировать, погибнет или выживет пассажир Титаника
# Целевой признак - Survived


+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Family_Size|Alone|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|          1|    0|
|       1|     1|female|38.0|    1|    0|71.2833|       C|          1|    0|
|       1|     3|female|26.0|    0|    0|  7.925|       S|          0|    1|
|       1|     1|female|35.0|    1|    0|   53.1|       S|          1|    0|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|          0|    1|
|       0|     3|  male|30.0|    0|    0| 8.4583|       Q|          0|    1|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|          0|    1|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|          4|    0|
|       1|     3|female|27.0|    0|    2|11.1333|       S|          2|    0|
|       1|     2|female|14.0|    1|    0|30.0708|       C|          1|    0|

In [ ]:
# Поработаем с категориальными признаками.

#Функция StringIndexer индексирует (энкодит) строки
#Концепция, лежащая в основе индексирования строк, очень интуитивно понятна.
#Мы просто заменяем каждую категорию номером.

sex_index = StringIndexer(inputCol='Sex', outputCol="Sex_index")  #StringIndexer - это Estimator, который нам формирует Transformer для преобразования данных
embarked_index = StringIndexer(inputCol='Embarked', outputCol="Embarked_index")

In [ ]:
# теперь применим новые столбцы к нашему датафрейму. Сначала обращаемся ним и выполняем операцию fit на нашем датафрейме.
#После чего мы получим функцию трансформер и сможем вызвать ф-ию transform для того, чтобы выполнить преобразование наших данных

titanic_df = sex_index.fit(titanic_df).transform(titanic_df)



In [ ]:
titanic_df = embarked_index.fit(titanic_df).transform(titanic_df)

In [ ]:
# Сделайте преобразование для embarked_index


In [ ]:
titanic_df.show()

+--------+------+------+----+-----+-----+-------+--------+-----------+-----+---------+--------------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Family_Size|Alone|Sex_index|Embarked_index|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+---------+--------------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|          1|    0|      0.0|           0.0|
|       1|     1|female|38.0|    1|    0|71.2833|       C|          1|    0|      1.0|           1.0|
|       1|     3|female|26.0|    0|    0|  7.925|       S|          0|    1|      1.0|           0.0|
|       1|     1|female|35.0|    1|    0|   53.1|       S|          1|    0|      1.0|           0.0|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|          0|    1|      0.0|           0.0|
|       0|     3|  male|30.0|    0|    0| 8.4583|       Q|          0|    1|      0.0|           2.0|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|          0|    1|      

In [ ]:
# Наш dataframe готов к тому, чтобы применить к нему модель. Но мы хотим сначала выполнить векторизацию, чтобы данные можно было применять в моделе.
# Формируем список признаков, которые мы будем использовать, как фичи.
features = ['Pclass', 'Age', 'SibSp', 'SibSp', 'Parch', 'Fare', 'Alone', 'Sex_index', 'Embarked_index']

In [ ]:
feature = VectorAssembler(inputCols=features, outputCol="features")
#VectorAssembler объединяет заданный список столбцов в один векторный столбец.
# Является, пожалуй, самым важным векторным преобразователем в PySpark,
# поскольку модели машинного обучения требуют на вход векторы.

# VectorAssembler - трансформер, соответственно, вызываем метод transform  и применяем его к нашему датафрейму.
feature_vector= feature.transform(titanic_df)

In [ ]:
feature_vector.show(5)

+--------+------+------+----+-----+-----+-------+--------+-----------+-----+---------+--------------+--------------------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Family_Size|Alone|Sex_index|Embarked_index|            features|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+---------+--------------+--------------------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|          1|    0|      0.0|           0.0|[3.0,22.0,1.0,1.0...|
|       1|     1|female|38.0|    1|    0|71.2833|       C|          1|    0|      1.0|           1.0|[1.0,38.0,1.0,1.0...|
|       1|     3|female|26.0|    0|    0|  7.925|       S|          0|    1|      1.0|           0.0|[3.0,26.0,0.0,0.0...|
|       1|     1|female|35.0|    1|    0|   53.1|       S|          1|    0|      1.0|           0.0|[1.0,35.0,1.0,1.0...|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|          0|    1|      0.0|           0.0|(9,[0,1,5,6],[3.0...|
+--------+------

In [ ]:
(training_data, test_data) = feature_vector.randomSplit([0.8, 0.2],seed = 42)
#Формируем тренировочный и тестовый датасет. 80% данных в тренировочный, остальные - в тестовый. Seed - параметр случайности

In [ ]:
training_data.show(5)

+--------+------+------+----+-----+-----+------+--------+-----------+-----+---------+--------------+--------------------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|  Fare|Embarked|Family_Size|Alone|Sex_index|Embarked_index|            features|
+--------+------+------+----+-----+-----+------+--------+-----------+-----+---------+--------------+--------------------+
|       0|     1|female| 2.0|    1|    2|151.55|       S|          3|    0|      1.0|           0.0|[1.0,2.0,1.0,1.0,...|
|       0|     1|female|25.0|    1|    2|151.55|       S|          3|    0|      1.0|           0.0|[1.0,25.0,1.0,1.0...|
|       0|     1|  male|18.0|    1|    0| 108.9|       C|          1|    0|      0.0|           1.0|[1.0,18.0,1.0,1.0...|
|       0|     1|  male|19.0|    1|    0|  53.1|       S|          1|    0|      0.0|           0.0|[1.0,19.0,1.0,1.0...|
|       0|     1|  male|19.0|    3|    2| 263.0|       S|          5|    0|      0.0|           0.0|[1.0,19.0,3.0,3.0...|
+--------+------+------+

# ML models

# LogisticRegression

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

#Сначала создаем объект evaluator. Указываем целевую колонку обучения модели - labelCol. далее указываем
# имя колонки, где будет лежать предсказанное значение  - predictionCol. И указываем метрику для оценки качества модели.

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="Survived", featuresCol="features") # labelCol="Survived" - целевая фича,  featuresCol="features" - фичи, которые
#используются для предсказания занчения целевой колонки

lrModel = lr.fit(training_data) # получаем обученную модель.

lr_prediction = lrModel.transform(test_data) # применяем модель на данных, получаем предсказание
lr_prediction.select("prediction", "Survived", "features").show(5)

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       1.0|       0|[1.0,50.0,0.0,0.0...|
|       1.0|       0|(9,[0,1,4,5],[1.0...|
|       1.0|       0|[1.0,24.0,0.0,0.0...|
|       0.0|       0|(9,[0,1,5,6],[1.0...|
|       0.0|       0|(9,[0,1,5,6],[1.0...|
+----------+--------+--------------------+
only showing top 5 rows



In [ ]:
# Применяем оценщик (evaluator), чтобы узнать точность модели.
lr_accuracy = evaluator.evaluate(lr_prediction)
print("LogisticRegression [Accuracy] = %g"% (lr_accuracy))
print("LogisticRegression [Error] = %g " % (1.0 - lr_accuracy))

LogisticRegression [Accuracy] = 0.813793
LogisticRegression [Error] = 0.186207 


# DecisionTreeClassifier

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="Survived", featuresCol="features")
dt_model = dt.fit(training_data)
dt_prediction = dt_model.transform(test_data)

dt_prediction.select("prediction", "Survived", "features").show(5)

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       1.0|       0|[1.0,50.0,0.0,0.0...|
|       0.0|       0|(9,[0,1,4,5],[1.0...|
|       0.0|       0|[1.0,24.0,0.0,0.0...|
|       0.0|       0|(9,[0,1,5,6],[1.0...|
|       0.0|       0|(9,[0,1,5,6],[1.0...|
+----------+--------+--------------------+
only showing top 5 rows



In [ ]:
dt_accuracy = evaluator.evaluate(dt_prediction)
print("DecisionTreeClassifier [Accuracy] = %g"% (dt_accuracy))
print("DecisionTreeClassifier [Error] = %g " % (1.0 - dt_accuracy))

DecisionTreeClassifier [Accuracy] = 0.82069
DecisionTreeClassifier [Error] = 0.17931 


# RandomForestClassifier

Реализуйте самостоятельно и оцените точность, как в примерах выше

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

# Define Random Forest classifier with labelCol and featuresCol specified
rf = RandomForestClassifier(labelCol="Survived", featuresCol="features")

# Train the Random Forest classifier on the training data
rf_model = rf.fit(training_data)

# Make predictions on the test data using the trained model
rf_prediction = rf_model.transform(test_data)


rf_prediction.select("prediction", "Survived", "features").show(5)


# Evaluate the Random Forest classifier model using the MulticlassClassificationEvaluator
rf_accuracy = evaluator.evaluate(rf_prediction)

# Print the accuracy and error of the Random Forest classifier
print("RandomForestClassifier [Accuracy] = %g" % rf_accuracy)
print("RandomForestClassifier [Error] = %g" % (1.0 - rf_accuracy))


+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       1.0|       0|[1.0,50.0,0.0,0.0...|
|       1.0|       0|(9,[0,1,4,5],[1.0...|
|       0.0|       0|[1.0,24.0,0.0,0.0...|
|       0.0|       0|(9,[0,1,5,6],[1.0...|
|       0.0|       0|(9,[0,1,5,6],[1.0...|
+----------+--------+--------------------+
only showing top 5 rows

RandomForestClassifier [Accuracy] = 0.813793
RandomForestClassifier [Error] = 0.186207


# Gradient-boosted tree classifier

Реализуйте самостоятельно и оцените точность, как в примерах выше

In [ ]:
from pyspark.ml.classification import GBTClassifier

# Define Gradient-Boosted Tree classifier with labelCol and featuresCol specified
gbt = GBTClassifier(labelCol="Survived", featuresCol="features")

# Train the Gradient-Boosted Tree classifier on the training data
gbt_model = gbt.fit(training_data)

# Make predictions on the test data using the trained model
gbt_prediction = gbt_model.transform(test_data)

gbt_prediction.select("prediction", "Survived", "features").show(5)

# Evaluate the Gradient-Boosted Tree classifier model using the MulticlassClassificationEvaluator
gbt_accuracy = evaluator.evaluate(gbt_prediction)

# Print the accuracy and error of the Gradient-Boosted Tree classifier
print("Gradient-Boosted Tree Classifier [Accuracy] = %g" % gbt_accuracy)
print("Gradient-Boosted Tree Classifier [Error] = %g" % (1.0 - gbt_accuracy))


+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       1.0|       0|[1.0,50.0,0.0,0.0...|
|       0.0|       0|(9,[0,1,4,5],[1.0...|
|       0.0|       0|[1.0,24.0,0.0,0.0...|
|       1.0|       0|(9,[0,1,5,6],[1.0...|
|       1.0|       0|(9,[0,1,5,6],[1.0...|
+----------+--------+--------------------+
only showing top 5 rows

Gradient-Boosted Tree Classifier [Accuracy] = 0.862069
Gradient-Boosted Tree Classifier [Error] = 0.137931


# Save & Load Model

In [ ]:
# обученную модель мы можем сохранять. Сохраним модель RandomForest.
rf_model.write().overwrite().save('rf_model')

In [ ]:
# Теперь модель загрузим. Для этого сначала модключим нужный класс
from pyspark.ml.classification import RandomForestClassificationModel
type(RandomForestClassificationModel.load('rf_model'))

pyspark.ml.classification.RandomForestClassificationModel

# Pipeline

In [ ]:
from pyspark.ml.pipeline import PipelineModel

# pipeline упрощает процесс внедрения ваших моделей в нужное окружение.

# Перед тем, как модель обучить, мы уже выполнили какие-то преобразования данных.
# Когда мы модель внедряем, нам нужно внедрять и все преобразования над данными.
# Соответственно, появляется место для ошибок (человеческий фактор). Какую-то предобработку могут забыть сделать или сделать неправильно.
# Пайплайны позволяют собрать вашу модель вместе с трансформациями.

In [ ]:
titanic_df = spark.read.parquet('train.parquet')

In [ ]:
titanic_df.show()

+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Family_Size|Alone|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|          1|    0|
|       1|     1|female|38.0|    1|    0|71.2833|       C|          1|    0|
|       1|     3|female|26.0|    0|    0|  7.925|       S|          0|    1|
|       1|     1|female|35.0|    1|    0|   53.1|       S|          1|    0|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|          0|    1|
|       0|     3|  male|30.0|    0|    0| 8.4583|       Q|          0|    1|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|          0|    1|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|          4|    0|
|       1|     3|female|27.0|    0|    2|11.1333|       S|          2|    0|
|       1|     2|female|14.0|    1|    0|30.0708|       C|          1|    0|

In [ ]:
train, test = titanic_df.randomSplit([0.8, 0.2])

In [ ]:
train.show(5)

+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Family_Size|Alone|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|       0|     1|female| 2.0|    1|    2| 151.55|       S|          3|    0|
|       0|     1|female|25.0|    1|    2| 151.55|       S|          3|    0|
|       0|     1|female|50.0|    0|    0|28.7125|       C|          0|    1|
|       0|     1|  male|18.0|    1|    0|  108.9|       C|          1|    0|
|       0|     1|  male|19.0|    1|    0|   53.1|       S|          1|    0|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
only showing top 5 rows



In [ ]:
indexer_sex = StringIndexer(inputCol="Sex", outputCol="Sex_index")

In [ ]:
indexer_embarked = StringIndexer(inputCol="Embarked", outputCol="Embarked_index")

In [ ]:
feature = VectorAssembler(
    inputCols=["Pclass","Age","SibSp","Parch","Fare","Family_Size","Embarked_index","Sex_index"],
    outputCol="features")


In [ ]:
rf_classifier = RandomForestClassifier(labelCol="Survived", featuresCol="features")

In [ ]:
pipeline = Pipeline(stages=[indexer_sex, indexer_embarked, feature, rf_classifier])
# Определились с моделью созддаём Pipeline. Указываем массив из трансформаций, которые необходимо выполнить.
# Собранный Pipeline является Estimator'ом, вызываем метод fit, из датафрейма (начального) в одно действие получается готовая модель.

In [ ]:
p_model = pipeline.fit(train)

In [ ]:
type(p_model)

pyspark.ml.pipeline.PipelineModel

In [ ]:
p_model.write().overwrite().save('p_model')
# сохраним модель с перезаписью.

In [ ]:
model = PipelineModel.load('p_model')
# Загрузим модель.

In [ ]:
prediction = p_model.transform(test)
# применим модель, вызовем метод transrorm, применим его к выборке test

In [ ]:
test.show(5)

+--------+------+----+----+-----+-----+-------+--------+-----------+-----+
|Survived|Pclass| Sex| Age|SibSp|Parch|   Fare|Embarked|Family_Size|Alone|
+--------+------+----+----+-----+-----+-------+--------+-----------+-----+
|       0|     1|male|30.0|    0|    0| 25.925|       S|          0|    1|
|       0|     1|male|30.0|    0|    0|27.7208|       C|          0|    1|
|       0|     1|male|30.0|    0|    0|30.6958|       C|          0|    1|
|       0|     1|male|30.0|    0|    0|   31.0|       S|          0|    1|
|       0|     1|male|30.0|    0|    0|   39.6|       C|          0|    1|
+--------+------+----+----+-----+-----+-------+--------+-----------+-----+
only showing top 5 rows



In [ ]:
prediction.select(["Pclass","Age","SibSp","Parch","Fare","Family_Size","Embarked_index","Sex_index","prediction"]).show(5)

+------+----+-----+-----+-------+-----------+--------------+---------+----------+
|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Embarked_index|Sex_index|prediction|
+------+----+-----+-----+-------+-----------+--------------+---------+----------+
|     1|30.0|    0|    0| 25.925|          0|           0.0|      0.0|       0.0|
|     1|30.0|    0|    0|27.7208|          0|           1.0|      0.0|       0.0|
|     1|30.0|    0|    0|30.6958|          0|           1.0|      0.0|       0.0|
|     1|30.0|    0|    0|   31.0|          0|           0.0|      0.0|       0.0|
|     1|30.0|    0|    0|   39.6|          0|           1.0|      0.0|       0.0|
+------+----+-----+-----+-------+-----------+--------------+---------+----------+
only showing top 5 rows



In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

In [ ]:
p_accuracy = evaluator.evaluate(prediction)
print("Pipeline model [Accuracy] = %g"% (p_accuracy))
print("Pipeline model [Error] = %g " % (1.0 - p_accuracy))

Pipeline model [Accuracy] = 0.772727
Pipeline model [Error] = 0.227273 


Сделайте самостоятельно Pipeline для Градиентного Бустинга, проверьте точность на тестовой выборке.

In [ ]:
titanic_df = spark.read.parquet('train.parquet')
titanic_df.show()

+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Family_Size|Alone|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|          1|    0|
|       1|     1|female|38.0|    1|    0|71.2833|       C|          1|    0|
|       1|     3|female|26.0|    0|    0|  7.925|       S|          0|    1|
|       1|     1|female|35.0|    1|    0|   53.1|       S|          1|    0|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|          0|    1|
|       0|     3|  male|30.0|    0|    0| 8.4583|       Q|          0|    1|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|          0|    1|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|          4|    0|
|       1|     3|female|27.0|    0|    2|11.1333|       S|          2|    0|
|       1|     2|female|14.0|    1|    0|30.0708|       C|          1|    0|

In [ ]:
train_data, test_data = titanic_df.randomSplit([0.8, 0.2])
train_data.show(5)

+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Family_Size|Alone|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|       0|     1|female| 2.0|    1|    2| 151.55|       S|          3|    0|
|       0|     1|female|25.0|    1|    2| 151.55|       S|          3|    0|
|       0|     1|female|50.0|    0|    0|28.7125|       C|          0|    1|
|       0|     1|  male|18.0|    1|    0|  108.9|       C|          1|    0|
|       0|     1|  male|19.0|    1|    0|   53.1|       S|          1|    0|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
only showing top 5 rows



In [ ]:
indexed_sex = StringIndexer(inputCol="Sex", outputCol="Sex_index")

In [ ]:
indexed_embarked = StringIndexer(inputCol="Embarked", outputCol="Embarked_index")

In [ ]:
feature = VectorAssembler(
    inputCols=["Pclass","Age","SibSp","Parch","Fare","Family_Size","Embarked_index","Sex_index"],
    outputCol="features")

gbt_classifier = GBTClassifier(labelCol="Survived", featuresCol="features")

In [ ]:
pipeline = Pipeline(stages=[indexed_sex, indexed_embarked, feature, gbt_classifier])
a_model = pipeline.fit(train)
type(a_model)

pyspark.ml.pipeline.PipelineModel

In [ ]:
p_model.write().overwrite().save('a_model')
model2 = PipelineModel.load('a_model')

prediction2 = p_model.transform(test_data)
test_data.show(5)

+--------+------+----+----+-----+-----+--------+--------+-----------+-----+
|Survived|Pclass| Sex| Age|SibSp|Parch|    Fare|Embarked|Family_Size|Alone|
+--------+------+----+----+-----+-----+--------+--------+-----------+-----+
|       0|     1|male|19.0|    3|    2|   263.0|       S|          5|    0|
|       0|     1|male|24.0|    0|    1|247.5208|       C|          1|    0|
|       0|     1|male|28.0|    1|    0| 82.1708|       C|          1|    0|
|       0|     1|male|30.0|    0|    0|     0.0|       S|          0|    1|
|       0|     1|male|30.0|    0|    0|     0.0|       S|          0|    1|
+--------+------+----+----+-----+-----+--------+--------+-----------+-----+
only showing top 5 rows



In [ ]:
prediction2.select(["Pclass","Age","SibSp","Parch","Fare","Family_Size","Embarked_index","Sex_index","prediction"]).show(5)

+------+----+-----+-----+--------+-----------+--------------+---------+----------+
|Pclass| Age|SibSp|Parch|    Fare|Family_Size|Embarked_index|Sex_index|prediction|
+------+----+-----+-----+--------+-----------+--------------+---------+----------+
|     1|19.0|    3|    2|   263.0|          5|           0.0|      0.0|       0.0|
|     1|24.0|    0|    1|247.5208|          1|           1.0|      0.0|       0.0|
|     1|28.0|    1|    0| 82.1708|          1|           1.0|      0.0|       0.0|
|     1|30.0|    0|    0|     0.0|          0|           0.0|      0.0|       0.0|
|     1|30.0|    0|    0|     0.0|          0|           0.0|      0.0|       0.0|
+------+----+-----+-----+--------+-----------+--------------+---------+----------+
only showing top 5 rows



In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
p_accuracy = evaluator.evaluate(prediction2)
print("Pipeline model [Accuracy] = %g"% (p_accuracy))
print("Pipeline model [Error] = %g " % (1.0 - p_accuracy))

Pipeline model [Accuracy] = 0.865854
Pipeline model [Error] = 0.134146 
